<a href="https://colab.research.google.com/github/yeonheuiyeon/Machine-Learning-Practice/blob/master/0526_Lesson7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.MNIST CNN

Pillow(A.K.A. PIL)가 실제로 우리에게 그것을 열어주는 것이기 때문에 결국 pillow로 가게 되는 과정에서 통과시킬 수 있고, 이 경우 이것들은 RGB가 아닌 흑백이기 때문에 베개의 convert_mode='를 사용해야 한다.L'.  하지만 이건 그레이스케일이다. 

우리의 이미지 아이템 리스트에는 7만개의 아이템이 포함되어 있으며, 1 X 28 X 28의 이미지 묶음이다. PyTorch는 채널을 우선시하므로 하나의 채널 28x28이다.  모든 CONV2D 같은 것들은 3등급 텐더에서 작동하기 때문에, 당신은 시작에 그 단위 축을 포함하기를 원하기 때문에, fast.ai은 한 채널 이미지를 읽을 때 조차도 그렇게 한다.

label_from_folder 함수를 통해 데이터를 SPLIT한다.
Item 리스트를 만들고, split 으로 나누고, 라벨을 붙인다.

그다음으로, 변환을 추가하는 것이다. 이 경우, 우리는 일반적인 get_transforms 기능을 사용하지 않는다. 왜냐하면 우리는 숫자 인식과 숫자 인식을 하고 있기 때문이다. 당신은 그것을 오른쪽으로 뒤집기를 원하지 않을 것이다. 그것은 그것의 의미를 바꿀 것이다. 그렇게 하면 의미가 바뀔 것이다. 또한 이 이미지들은 너무 작기 때문에 줌과 같은 것을 하는 것은 그것들을 읽을 수 없을 정도로 흐릿하게 만들 것이다. 그래서 보통 이런 작은 숫자의 이미지들은 무작위 패딩을 약간만 추가하면 된다. 그래서 두 개의 변환을 실제로 돌려주는 임의 패딩 기능을 사용할 것이다. 패딩을 하는 비트와 랜덤 크롭을 하는 비트. 따라서 이 두 변환을 모두 이 목록에 넣으려면 별(*)을 사용해야 한다.

## 1-1) 간단한 cnn만들기
우선 간단한 CNN을 만들기. input은 28* 28이다.모든  콘볼루션은 커널 사이즈 3 스트레이드 2 패딩 1이 될 것이다. 그래서 간단한 기능을 만들기 한다.

첫 번째 콘볼루션 이후에 한 채널이 들어오는데, 한 채널이 있는 그레이스케일 이미지이다. 따라서 완전히 연결된 계층에 관계없이 얼마나 많은 필터를 생성할지 항상 선택해야 하며, 이 경우 매트릭스의 너비만 곱하거나, 2D 컨벤션을 사용하는 경우 필터 수를 선택할 수 있다. 그래서 나는 8을 선택했고, 그 후에 2에서 28까지 진행되었고, 그래서 28은 8개의 채널이 있는 14 X 14 피쳐 맵이 되었다. 구체적으로 말하자면, 8 X 14 X 14 10의 활성화가 된다.

그럼 일괄적으로 표준화하고, 그 다음에 ReLU를 사용한다. 다음 콘벤트에 들어가는 입력 필터의 수는 이전  conv 의 출력 필터의 수와 같아야 하고, 우리는 채널 수를 계속 늘릴 수 있다. 왜냐하면 우리는 스트레이드 2를 하고 있기 때문이다. 그것은 그리드 크기를 계속 줄여야 한다. 

Batch norm, ReLU, conv.  우리는 이제 2by 2로 떨어졌다.Batch norm, ReLU, conv.  이제 1by 1로 줄어든다. 그 후, 우리는 10 X 1 X 1의 피쳐 맵을 가지고 있다. 그게 말이 돼? 지금 격자 크기가 하나야. 길이 10의 벡터가 아니라 3위 10, 1, 1의 텐서 입니다. 우리의 손실 함수는 (일반적으로) 3등급이 아닌 벡터를 기대하기 때문에, 마지막에 평평하게 처크할 수 있고, 평평하게 하는 것은 어떤 단위 축도 제거한다는 것을 의미한다. 그래서 그것은 이제 우리가 항상 기대하는 길이 10의 벡터일 뿐이다.

그래야 CNN을 만들 수 있다. 그런 다음 데이터와 모델, 손실 기능 및 선택적으로 일부 측정 기준을 제출하여 학습자에게 반환할 수 있다. 우리는 평상시처럼 교차 엔트로피를 사용할 것이다. 그러면 learn.summary()로 전화해서 확인할 수 있다.

2) crappify 로직

- 멋진 이미지를 열어본다.
- bilinear interpolation 으로 96 x 96 픽셀의 작은 크기로 크기를 조정한다.
-  10에서 70 사이의 임의의 숫자를 선택한다.
- 임의의 위치에 있는 내 이미지에 그 번호를 그려 넣는다.
- 그 임의의 숫자의 JPEG 화질로 그 이미지를 저장한다.

# 2. Gan
진행 방식 : Crappy image -> Generator -> 예측과 함께 pixel MSE와 hi-res image 그리고 최종적으로 Discriminator/Critic 단계로 마무리

Discriminator/Critic생성법
 image_gen이라는 디렉토리를 만들어 path_gen이라는 변수에 담아 낸다. save_preds라는 작은 기능을 가지고 있는데, 이 기능은 데이터 로더를 필요로 한다. 파일명을 다 잡는다. 항목 목록에 .items가 이미지 항목 목록인 경우 파일 이름을 포함한다는 점을 기억해야한다. 그리고 그 배치에 대한 예측을 한 묶음으로 가져와서 재구성한다.사실 이것은 실제로 fast.ai 이미지 오브젝트를 일괄적으로 만들 것이라는 것을 의미한다. 그러면 우리는 그 예측들을 하나하나 검토해서 저장해 둘 것이다. 우리가 그것을 저장할 이름은 원본 파일의 이름이지만, 우리는 그것을 우리의 새로운 디렉토리에 넣을 것이다.

예측을 저장해 두는 것이다. 보시다시피, 나는 이미 fast.ai 라이브러리에 있는 것만 사용하는 것이 아니라, 여러분이 직접 어떻게 글을 쓰는지 보여주려고 노력한다. 그리고 일반적으로 그것을 하기 위해 많은 코드를 필요로 하지 않는다. 

# 3. Recurrent Neural Network (RNN)
직사각형은 입력을 의미한다. 입력 수에 따른 일괄 처리 크기다. 화살표는 ReLU 다음에 나오는 매트릭스 제품과 같은 레이어(광범위하게 정의됨)를 의미한다. 원은 활성화다. 이 경우, 우리는 숨겨진 활성화의 한 집합을 가지고 있으며, 따라서 입력이 입력의 수였다는 점을 감안할 때, 여기(첫 번째 화살표)는 입력의 수에 따른 입력의 행렬이다. 따라서 출력은 여러 활성화에 의해 배치 크기가 될 것이다.

이 모양들을 어떻게 계산하는지 아는 것이 정말 중요하다. 그러니 가서 많은 것을 배워라.요약()은 모든 모양을 볼 수 있다. 그리고 여기 또 다른 화살표가 있는데, 이것은 그것이 또 다른 층이라는 것을 의미한다; 매트릭스 제품 다음에 비선형성이 뒤따른다. 이 경우에는 출력물에 들어가므로 소프트맥스를 사용한다.

그러면 삼각형은 출력을 의미한다. 이 매트릭스 제품은 여러 클래스에 의한 활성화 수가 될 것이므로, 우리의 출력은 숫자 클래스별 배치 크기다.

#4. Human numbers 
이 경우 검증 세트는 8000 이상의 숫자로, 교육 세트는 1~8000이다. 우리는 그것들을 합쳐서 데이터 묶음으로 만들 수 있다.

우리가 요청한 배치 크기는 64이고, 그리고 기본적으로 70.bpt라고 불리는 것을 사용한다. bptt는 우리가 간단히 언급했듯이, "시간을 통한 후방 지원"을 의미한다. 그게 바로 시퀀스 길이야. 64개의 문서 세그먼트를 각각 한 번에 살펴보는 70개의 단어 목록으로 나누었다. 그래서 검증 세트를 위해 13,000개의 토큰으로 구성된 전체 문자열을 가져다가 64개의 동일한 크기의 섹션으로 분할하는 겁니다. 사람들은 종종 내가 뭔가 다른 말을 하고 있다고 생각한다. 나는 그들이 길이가 64세라고 말하지 않았다 - 그들은 아니다. 그것들은 대략 같은 크기의 64개 입니다. 그래서 우리는 문서 1의 1/64를 가져간다. 

그리고 그 문서들의 1/64에 대해, 우리는 그것들을 길이 70의 조각으로 나누었다. 그럼 이제 그 1만 3천개의 토큰에 대해 몇 개의 묶음이 있다.  배치 사이즈로 나누고 70으로 나누면 3개 배치로 나누어진다.

내용에는 1개의 임베딩(즉, 녹색 화살표), 1개는 숨겨진 갈색 화살표 레이어에, 1개는 출력물에 숨겨진 화살표 레이어를 포함한다. 그래서 각각의 색깔 화살표는 하나의 행렬을 가지고 있다. 그런 다음 전진 패스에서는 첫 번째 입력 x[0]를 가지고 숨겨진 입력(녹색 화살표)에 입력하여 h라고 부르는 첫 번째 활성화 세트를 만든다. 두 번째 단어가 있다고 가정하면, 때때로 우리는 두 번째 단어가 없는 한 묶음의 끝에 있을 수 있기 때문이다. 두 번째 단어가 있다고 가정하면 녹색 화살표(i_h)를 통해 x[1]를 입력한 결과를 h에 추가할 수 있다. 그리고 나서 우리는 "좋아, 우리의 새로운 h는 그 두 사람이 합쳐져서 숨겨진 우리의 숨겨진 화살표(주황색 화살표)를 뚫고, 그리고 나서 RelLU는 배치 규범에 의해 만들어진 결과물이야. 그리고 나서 두 번째 단어로, 정확히 같은 것을 하라. 그리고 마지막으로 파란색 화살표 - h_o로 연결한다.

파이참으로 디버깅을 진행하면서 살폈던 
step in, out 과정들과 변수들 확인 캡쳐 내용첨부
https://drive.google.com/open?id=1DCn7NA2DAVvxK8m8EWeG3riHCR5HDs58